Here we find the extrinsics of our camera calibration w.r.t the RV calibration board using the RV calibration.

In [ ]:
from pathlib import Path
import sys

import matplotlib.pyplot as plt
import torch

sys.path.append('/home/universal/Downloads/dev.sk_robot_rgbd_data/src')
from skrgbd.calibration.calibrations.small_scale_sphere import Calibration
from skrgbd.calibration.camera_models.central_generic import CentralGeneric
from skrgbd.calibration.camera_models.rv_camera_model import fit_camera_model, RVCameraModel

## Fit the transform

Scan something with texture, for instance, our calibration board.

In [ ]:
scan_dir = Path(
    '/home/universal/Downloads/dev.sk_robot_rgbd_data/stl_shared_folder/scans/test_calib_board_folder'
)
scan_id = 0
rv_camera_model = RVCameraModel(f'{scan_dir}/scan_res_{scan_id:04}/Raw/imparb01.txt')

calib_dir = Calibration.calib_dir
our_camera_model = CentralGeneric(f'{calib_dir}/stl_right_intrinsics.yaml',
                                  f'{calib_dir}/stl_right_inverse_grid.pt')

Fit the transform.

In [ ]:
rotation, translation, residuals = fit_camera_model(
    our_camera_model, rv_camera_model,
    samples_n=1_000_000,
    depth_range=(.1, 2),
    verbose=2
)

Check that the residuals are small.

In [ ]:
_ = 4
plt.figure(figsize=(4 * _, _))
plt.hist(residuals, 1000);

Save the transform to disk.

In [ ]:
rv_calib_to_stl_right = _ = torch.zeros(4, 4)
_[3, 3] = 1
_[:3, :3] = rotation
_[:3, 3] = translation
torch.save(rv_calib_to_stl_right, f'{calib_dir}/rv_calib_to_stl_right.pt')

## Check the transform.

In RV ScanCenter there are several different coordinate spaces:
* camera space,
* the space of the calibration board,
* the "mesh space",
* the world space.


* The `Raw/impara01.txt` and `Raw/imparb01.txt` define the transform between the camera space and the space of the calibration board.
* The transform from the space of the calibration board to the "mesh space" is defined in `vertex_matrix.txt`.
* The transform from the "mesh space" to the world space is defined in `Raw/M_scenemesh.bin`

When you export in `ply` format, the scan is saved in the "mesh space",
and when you export in any other format (in fact, checked only for `obj`), the scan is saved in the world space.

In [ ]:
from PIL import Image
import numpy as np
import open3d as o3d

In [ ]:
scan = f'{scan_dir}/test_calib_board/Mesh_{scan_id:04}/Mesh_{scan_id:04}.obj'
scenemesh = f'{scan_dir}/scan_res_{scan_id:04}/Raw/M_scenemesh.bin'
vertex_matrix =  f'{scan_dir}/scan_res_{scan_id:04}/vertex_matrix.txt'
texture_left = f'{scan_dir}/scan_res_{scan_id:04}/Debug/Pictures/cama_maxwhite_00_0000.bmp'
texture_right = f'{scan_dir}/scan_res_{scan_id:04}/Debug/Pictures/camb_maxwhite_00_0000.bmp'

scan = o3d.io.read_triangle_mesh(scan)

texture_left = Image.open(texture_left)
texture_left = np.asarray(texture_left)
h, w = texture_left.shape

texture_right = Image.open(texture_right)
texture_right = np.asarray(texture_right)

scenemesh = np.fromfile(scenemesh, dtype=np.float32).reshape(4, 4)
scenemesh = torch.from_numpy(scenemesh).double()
mesh_to_world = scenemesh

vertex_matrix = np.loadtxt(vertex_matrix)
vertex_matrix = torch.from_numpy(vertex_matrix).double()
board_to_mesh = vertex_matrix

In [ ]:
rv_calib_to_stl_right = torch.load(f'{calib_dir}/rv_calib_to_stl_right.pt')

### Check in 3D

Transfer texture coordinates from the triangle vertices to the mesh vertices.

In [ ]:
triangles = np.asarray(scan.triangles)
triangle_uvs = np.asarray(scan.triangle_uvs)

triangles = torch.from_numpy(triangles).long()
triangle_uvs = torch.from_numpy(triangle_uvs)

vertex_uvs = torch.full([len(scan.vertices), 2], -1, dtype=torch.double)
vertex_uvs.scatter_(0, triangles.view(-1).unsqueeze(1).expand_as(triangle_uvs), triangle_uvs)

assert (vertex_uvs[triangles.ravel()] == triangle_uvs).all()

Sample colors from the texture, i.e photo from left STL camera.

In [ ]:
grid = vertex_uvs * 2 - 1
grid[:, 1] *= -1

colors = torch.nn.functional.grid_sample(
    torch.from_numpy(texture_left).double().div(255).unsqueeze(0).unsqueeze(1),
    grid.unsqueeze(0).unsqueeze(1),
    mode='bilinear'
).squeeze(2).squeeze(0)
colors = colors.T.clamp(0, 1).expand(-1, 3).contiguous()

Transform scan vertices to our camera space.

In [ ]:
scan_verts = np.asarray(scan.vertices)
scan_verts = torch.from_numpy(scan_verts.T).contiguous()

mm_to_meters = torch.zeros(4, 4)
mm_to_meters[3, 3] = 1
mm_to_meters[0, 0] = mm_to_meters[1, 1] = mm_to_meters[2, 2] = 1 / 1000

world_to_our_cam = rv_calib_to_stl_right @ mm_to_meters @ board_to_mesh.inverse() @ mesh_to_world.inverse()

scan_verts = world_to_our_cam[:3, :3] @ scan_verts + world_to_our_cam[:3, 3:4]

scan_verts = torch.nn.functional.normalize(scan_verts, dim=0)

In [ ]:
pc = o3d.geometry.PointCloud()
pc.points = o3d.utility.Vector3dVector(scan_verts.T.contiguous().numpy())
pc.colors = o3d.utility.Vector3dVector(colors.contiguous().numpy())
o3d.io.write_point_cloud('/tmp/verts.ply', pc)

Unproject pixels from our camera, i.e the right STL camera, to the camera space.

In [ ]:
v, u = torch.meshgrid(torch.arange(0, h, dtype=torch.double), torch.arange(0, w, dtype=torch.double))
uv = torch.stack([u.ravel() + .5, v.ravel() + .5]).contiguous()

scan_verts_from_texture = our_camera_model.unproject(uv)

In [ ]:
texture_colors = torch.from_numpy(texture_right).double().div(255).ravel().unsqueeze(1).expand(-1, 3)

In [ ]:
pc = o3d.geometry.PointCloud()

_ = scan_verts_from_texture.isfinite().all(0)
pc.points = o3d.utility.Vector3dVector(scan_verts_from_texture.T[_].contiguous().numpy())
pc.colors = o3d.utility.Vector3dVector(texture_colors[_].contiguous().numpy())
pc.remove_non_finite_points()
o3d.io.write_point_cloud('/tmp/verts_from_texture.ply', pc)

### Check in 2D

In [ ]:
sys.path.append('/home/universal/Downloads/dev.sk_robot_rgbd_data/debug/depth_map_reprojection_example/dev.mvs4df/src')
from mvs4df.modules.pointcloud_rendering.render_points import render_points

Project scan vertices in our camera space to the image space.

In [ ]:
device = 'cuda:0'

our_camera_model = our_camera_model.to(device)
uv = our_camera_model.project_fine(scan_verts.to(device)).cpu()
our_camera_model = our_camera_model.to('cpu')
torch.cuda.empty_cache()

Check the quality of the projection.

In [ ]:
_ = our_camera_model.unproject(uv)
_ = (_ - torch.nn.functional.normalize(scan_verts, dim=0)).norm(dim=0)
plt.hist(_.numpy(), 1000);
plt.yscale('log')

Render the projected points into the image.

In [ ]:
colors = colors.T.contiguous()

In [ ]:
w, h = our_camera_model.size_wh

render = render_points(
    colors.unsqueeze(0),
    scan_verts[2].unsqueeze(0),
    uv.unsqueeze(1),
    (h, w),
    point_radius=(2 ** -.5),
#     uv_averaging_range=1e-4,
    depth_averaging_range=1e-5,
).squeeze(0)

render = render.where(render.isfinite(), render.new_zeros([]))
img = render.permute(1, 2, 0).mul(255).clamp(0, 255).byte()

Compare the photo from the right camera with the rendering.

In [ ]:
Image.fromarray(img.numpy()).save('/tmp/render.png')
Image.fromarray(texture_right).save('/tmp/texture.png')